# Setup and Import Libraries

In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [2]:
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

# Embeddings

## OPENAI Embeddings

In [4]:
embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=3072)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000015B731361D0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000015B7F7A5650>, model='text-embedding-3-large', dimensions=3072, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [5]:
text = "Understanding OpenAI Embeddings"

In [6]:
query_result = embeddings.embed_query(text=text)
query_result

[0.005841143894940615,
 0.03672826290130615,
 -0.01822661980986595,
 -0.03860342875123024,
 0.004841055255383253,
 -0.002572103636339307,
 0.01602642424404621,
 0.02625233307480812,
 0.0032002844382077456,
 -0.022714518010616302,
 -0.01075095497071743,
 0.007775690872222185,
 -0.0037753356155008078,
 0.0022908286191523075,
 -0.012507361359894276,
 0.01551387831568718,
 0.02662736549973488,
 0.051404569298028946,
 -0.022927038371562958,
 -0.023939628154039383,
 -0.016951506957411766,
 -0.007650679908692837,
 -0.04122866317629814,
 -0.05735509842634201,
 0.025977307930588722,
 -0.03355298191308975,
 0.005087952129542828,
 0.02862754464149475,
 0.005491112824529409,
 0.05920526012778282,
 0.023202061653137207,
 -0.03452806919813156,
 0.01163228414952755,
 -0.010994726791977882,
 -0.02476469986140728,
 0.016626477241516113,
 0.042503777891397476,
 0.038553427904844284,
 -0.03885345160961151,
 0.031502798199653625,
 0.01761406473815441,
 0.003056521527469158,
 0.011219747364521027,
 0.00365

In [7]:
len(query_result)

3072

In [8]:
query_result[0]

0.005841143894940615

### Text Splitting and OpenAI Embeddings

In [9]:
loader = TextLoader('speech.txt')
docs = loader.load()
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness 

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_

### Vector Database

In [11]:
db = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)
db

### Query Vector Database

In [12]:
query = "It will be all the easier for us to conduct ourselves as belligerents"

In [13]:
retrieved_results = db.similarity_search(query=query)
retrieved_results

[Document(metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'s

In [14]:
retrieved_results[0].page_content

'It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early'

## Ollama

Ollama supports embedding models, making it possible to build Retrieval Augmented Generation (RAG) applications that combine text prompts with existing documents or other data

In [15]:
embeddings = (
    OllamaEmbeddings(model='gemma:2b')  ## By Default = llama2
)

In [16]:
r1 = embeddings.embed_documents(
    [
        "Alpha is the first letter of Greek Alphabet",
        "Beta is the second letter of Greek Alphabet"
    ]
)

r1

[[-0.040750574,
  -0.021344978,
  0.012801514,
  0.04282918,
  -0.0027583067,
  -0.00013498227,
  -0.007977057,
  -0.0034831553,
  0.0013797577,
  -0.013506961,
  0.0072624222,
  0.008170576,
  0.015068576,
  -0.013858002,
  -0.015085321,
  -0.003272624,
  0.051044483,
  -0.0062638405,
  0.014990707,
  0.0020610997,
  0.010718827,
  0.004586066,
  -0.00073094794,
  0.0077264295,
  -0.015282667,
  -0.012821573,
  -0.0077461824,
  0.0018204055,
  0.006972651,
  0.017472684,
  -0.0052282563,
  0.0031535123,
  0.0146112135,
  0.010991012,
  -0.019862585,
  -0.0008751137,
  -0.01583307,
  0.016264765,
  -0.0053084167,
  -0.004284598,
  0.021960009,
  0.00421616,
  0.011004659,
  -0.0061572227,
  -0.0014572757,
  -0.02284922,
  0.021677058,
  -0.006891316,
  -0.02138671,
  -0.0005694792,
  -0.27679828,
  -0.16519994,
  -0.0136389425,
  -0.0056997356,
  -0.02652083,
  -0.008636307,
  -0.011091062,
  0.008470114,
  -0.018202651,
  0.005888716,
  -0.0064210934,
  -0.016830325,
  -0.018565163,
 

In [17]:
len(r1[0])

2048

In [18]:
# r1[0]

In [19]:
query = "What is the second letter of Greek Alphabet"

embeddings.embed_query(text=query)

[-0.030105358,
 -0.005824135,
 0.0012754073,
 0.024623998,
 0.0012407452,
 0.00640913,
 -0.010951224,
 -0.0005147705,
 -0.009166065,
 -0.035265893,
 0.0005688289,
 -0.0020462004,
 0.018212501,
 -0.014931087,
 -0.0054999767,
 -0.004692976,
 0.11703483,
 -0.01323519,
 0.01449895,
 -0.0054749004,
 0.018045152,
 -0.014029271,
 0.008801458,
 0.007002419,
 -0.01408522,
 -0.019493215,
 0.00047266154,
 -0.0025420159,
 0.008156528,
 0.008699965,
 -0.0065714684,
 -0.0024327575,
 0.03203793,
 -0.00762293,
 -0.016872386,
 -0.005271813,
 -0.031765655,
 0.009123753,
 0.0002667427,
 -0.002819239,
 0.015168155,
 0.030735942,
 -0.010356899,
 -0.019818638,
 -0.009775126,
 -0.024147103,
 0.011634027,
 0.0016824671,
 -0.015575788,
 0.008889778,
 -0.26019627,
 -0.15498568,
 -0.017501716,
 -0.0051468955,
 -0.014350198,
 0.0047106235,
 -0.011944442,
 -0.00081661134,
 -0.006962856,
 -0.0023145285,
 -0.01867073,
 0.003568576,
 -0.012045148,
 -0.0107086,
 -0.021589026,
 -0.01672904,
 0.0045061354,
 -0.004685673

## HuggingFace Embeddings

In [20]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
text = "This is a text document"

In [22]:
query_result = embeddings.embed_query(text=text)

In [23]:
len(query_result)

384

In [24]:
# query_result